In [1]:
#!pip install SPARQLWrapper

In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON
import csv

In [3]:
LMDB_PREFIX = "<https://triplydb.com/Triply/linkedmdb/vocab/>"
SPARQL_ENDPOINT = "https://api.triplydb.com/datasets/Triply/linkedmdb/services/linkedmdb/sparql"
PROPERTY_NAMES = ["actor", "director", "genre", "cinematographer", "producer", "editor", "writer"] # must be from lmdb namespace

In [4]:
def get_movie_properties(movie):

    select_line = (" ").join(["?"+prop for prop in PROPERTY_NAMES])
    where_clause_lines = [f"Optional{{{movie} lmdb:{prop} ?{prop} . }}" for prop in PROPERTY_NAMES]
    where_clause = ("\n").join(where_clause_lines)

    query_str = f"""
    PREFIX lmdb: {LMDB_PREFIX}

    SELECT {select_line}
    WHERE {{
        {where_clause}
    }}
    """

    sparql = SPARQLWrapper(SPARQL_ENDPOINT)
    sparql.setQuery(query_str)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    properties = {prop: set() for prop in PROPERTY_NAMES}

    for result in results["results"]["bindings"]:
        for prop in PROPERTY_NAMES:
            if prop in result:
                properties[prop].add(result[prop]["value"])

    return properties

In [5]:
def get_movie_recommendations(properties, movie):    
    property_lines = []
    for prop, values in properties.items():
        for i, value in enumerate(values):
            property_lines.append(f"""
          {{
            ?movie lmdb:{prop} <{value}> .
            BIND("{prop}{i}" as ?matchedProperty)
          }}""")
    property_lines_combined = ("\nUNION\n").join(property_lines)

    query_str = f"""
    PREFIX lmdb: {LMDB_PREFIX}

    SELECT ?movie ?title (COUNT(DISTINCT ?matchedProperty) as ?score)
    WHERE {{
      ?movie a lmdb:Film .
      ?movie <http://purl.org/dc/terms/title> ?title .
      ?base_movie <http://purl.org/dc/terms/title> ?base_title
      {property_lines_combined}
      FILTER(?base_movie = {movie} && ?title != ?base_title)
    }}
    GROUP BY ?movie ?title
    ORDER BY DESC(?score)
    LIMIT 12
    """

    sparql = SPARQLWrapper(SPARQL_ENDPOINT)
    sparql.setQuery(query_str)
    sparql.setMethod('POST')
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    recommendations = []
    for result in results["results"]["bindings"]:
        movie_id = result["movie"]["value"].split('/')[-1]
        movie_title = result["title"]["value"]
        score = result["score"]["value"]
        recommendations.append({"movie_id": movie_id, "movie_title": movie_title, "score": score})

    return recommendations

In [6]:
def recommend_movies(movie_id):
    movie = f"<https://triplydb.com/Triply/linkedmdb/id/film/{movie_id}>"
    properties = get_movie_properties(movie)
    similar_movies = get_movie_recommendations(properties, movie)
    return similar_movies
        
def print_movies(movie_list):
    print("Similar Movies:")
    for movie in movie_list:
        print(f"Movie ID: {movie['movie_id']}, Movie Title: {movie['movie_title']}, Score: {movie['score']}")

In [7]:
similar_movies = recommend_movies(72)
print_movies(similar_movies)

Similar Movies:
Movie ID: 17780, Movie Title: Titanic Special Edition DVD, Score: 23
Movie ID: 33, Movie Title: Terminator 2: Judgment Day, Score: 8
Movie ID: 6220, Movie Title: Avatar, Score: 7
Movie ID: 15605, Movie Title: Revolutionary Road, Score: 7
Movie ID: 556, Movie Title: Aliens, Score: 6
Movie ID: 38324, Movie Title: True Lies, Score: 6
Movie ID: 2235, Movie Title: Strange Days, Score: 5
Movie ID: 16901, Movie Title: Great Expectations, Score: 4
Movie ID: 336, Movie Title: Apollo 13, Score: 4
Movie ID: 532, Movie Title: Romeo + Juliet, Score: 4
Movie ID: 1033, Movie Title: Tombstone, Score: 4
Movie ID: 8869, Movie Title: A Night to Remember, Score: 4


In [8]:
movie_list = []

with open('data/available_src_movies.csv') as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        movie_list.append(row[0])

In [9]:
## Getting Movie IDs
## Problem: Some movies and directors have multiple ids.
## For movies, they often have one id >= 38,000 and one < 38,000
## Tentative solution: Only consider ids < 38,000
def get_movie_id(movie_title):
    query_str = f"""
    PREFIX lmdb: {LMDB_PREFIX}

    SELECT ?movie
    WHERE {{
      ?movie a lmdb:Film .
      ?movie <http://purl.org/dc/terms/title> "{movie_title}" .
    }}
    """
    
    sparql = SPARQLWrapper(SPARQL_ENDPOINT)
    sparql.setQuery(query_str)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    movie_ids = []
    for result in results["results"]["bindings"]:
        movie_id = result["movie"]["value"].split('/')[-1]
        if (int(movie_id) >= 38000):
            continue
        movie_ids.append(movie_id)

    return movie_ids

In [10]:
movie_id_dic = {}

for movie in movie_list:
    found_ids = get_movie_id(movie)
    if (len(found_ids) == 1):
        movie_id_dic[movie] = found_ids[0]
    else:
        print(f"Manual correction required for '{movie}' with ids: {found_ids}.")
        movie_id_dic[movie] = found_ids

Manual correction required for 'Rebecca' with ids: ['11611', '2141'].
Manual correction required for 'Cool Hand Luke' with ids: ['27445', '8170'].
Manual correction required for 'Ben-Hur' with ids: ['2243', '2245'].
Manual correction required for 'Metropolis' with ids: ['2737', '14204', '63'].
Manual correction required for 'Heat' with ids: ['20088', '4657', '53', '9200'].


In [11]:
## Manual correction
movie_id_dic["Rebecca"] = '2141'
movie_id_dic["Cool Hand Luke"] = '27445'
movie_id_dic["Ben-Hur"] = '2245'
movie_id_dic["Metropolis"] = '63'
movie_id_dic["Heat"] = '53'

In [15]:
## Create recommendations and write in csv file

with open('data/available_src_movies_recs.csv', 'w', newline='') as file:
    writer = csv.writer(file, delimiter=',')
    for movie_title, movie_id in movie_id_dic.items():
        recs = recommend_movies(movie_id)
        line = [movie_title]
        for rec in recs:
            line.append(rec['movie_title'])
        writer.writerow(line)
    print("Finished")

Finished
